In [ ]:
import nltk
import numpy as np
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import random
import os


In [ ]:
stemmer=PorterStemmer()

def tokenize(query):
    return nltk.word_tokenize(query)

def stem(word):
    return stemmer.stem(word.lower())

def bags(tokenized,word_list):
    tokens=[stem(w) for w in tokenized]
    bag=np.zeros(len(word_list), dtype=np.float32)

    for i in range(len(word_list)):
        if word_list[i] in tokens:
            bag[i]=1.0

    return bag



In [ ]:
#Preparing the Training Data

desc = open('intents.json','r') #tags,patterns,responses
data=json.load(desc)

words_list=[]
tags=[]
training_data=[]

for intent in data['intents']:
    tag=intent['tag']
    tags.append(tag)

    for pattern in intent['patterns']:
      w=tokenize(pattern)
      words_list.extend(w) #Keeping Track of all the words for bag-of-words
      training_data.append((w,tag)) #X-Y Training Data

punctuations=['?','!','.','?']
buff=[]

trainx=[]
trainy=[]

buff=[stem(x) for x in words_list if x not in punctuations]

words_list=sorted(set(buff))
tags=sorted(set(tags))

for (tokens,tag) in training_data:
    trainx.append(bags(tokens,words_list)) #X -> Bag Of Words
    trainy.append(tags.index(tag)) #Y -> Tags for Each Sentence

trainx=np.array(trainx)
trainy=np.array(trainy)


#Create Dataset Loader
class CreateData(Dataset):
  def __init__(self):
    self.samples=len(trainx)
    self.datax=trainx
    self.datay=trainy

  def __getitem__(self,index):
    return self.datax[index],self.datay[index]
  
  def __len__(self):
    return self.samples


dataset1=CreateData()
loader=DataLoader(dataset=dataset1, batch_size=8,shuffle=True,num_workers=2)


#Creating a Class For Neural Network Model
class Model(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes):
    super(Model,self).__init__()
    self.l1=nn.Linear(input_size,hidden_size)
    self.l2=nn.Linear(hidden_size,hidden_size)
    self.l3=nn.Linear(hidden_size,num_classes)
    self.relu=nn.ReLU()

  
  def forward(self,x):
    out=self.l1(x)
    out=self.relu(out)
    out=self.l2(out)
    out=self.relu(out)
    out=self.l3(out)
    out=self.relu(out)

    return out


#Setting Epochs
epochs=1000

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=Model(len(words_list),8,(len(tags)))

loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

for epoch in range(1000):
  for(x,y) in loader:
    words=x.to(device)
    labels=y.to(device)

    outputs=model(words)
    lossy=loss(outputs,labels)

    optimizer.zero_grad()
    lossy.backward()
    optimizer.step()


  if (epoch+1)%100==0:
    print(f'epoch{epoch+1}/{epochs}, loss={lossy.item()}')

print(f'final loss={lossy.item()}')
model.eval()




epoch100/1000, loss=1.1042413711547852
epoch200/1000, loss=0.5161978602409363
epoch300/1000, loss=0.5220367312431335
epoch400/1000, loss=0.013459883630275726
epoch500/1000, loss=0.003509614383801818
epoch600/1000, loss=0.9743396043777466
epoch700/1000, loss=0.0009140105685219169
epoch800/1000, loss=0.4873718321323395
epoch900/1000, loss=0.48675110936164856
epoch1000/1000, loss=0.973254919052124
final loss=0.973254919052124


Model(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [ ]:
#Main Driver

print("How may we help you :) (type 'quit' to exit)")
learning=[]
while True:
  sentence = input("You: ")
  if sentence == "quit":
    answ=os.path.exists("learning.txt")
    with open("learning.txt", "a" if answ else "w") as f:
      f.writelines(learning)    
      break

  sent1 = tokenize(sentence)
  input_val = bags(sent1, words_list)
  input_val = input_val.reshape(1, input_val.shape[0])
  input_val = torch.from_numpy(input_val).to(device)

  output = model(input_val)
  _, predicted = torch.max(output, dim=1)

  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob=probs[0][predicted.item()]
   
    
  if prob.item() > 0.65:
      for intent in data['intents']:
          if tag == intent["tag"]:
              print(f"Sam: {random.choice(intent['responses'])}")
  else:
    learning.append(sentence+'\n')
    print("Sam: Sorry, I can't understand but your response is recorded and will be responded asap :)")
   

In [ ]:
import json
 
 
#Self Leaning Mechanism
def write_json(new_data, filename='intents.json'):
  check=False
  with open(filename,'r+') as file:
      file_data = json.load(file)
      for intent in file_data['intents']:
        if intent['tag']==new_data['tag']:
          check=True
          intent['patterns'].extend(new_data['patterns'])
          intent['responses'].extend(new_data['responses'])
        # Sets file's current position at offset.
      
      if check==False:
        file_data['intents'].append(new_data)
        

      file.seek(0)
        # convert back to json.
      json.dump(file_data, file, indent = 4)


c=""
dict1={}
with open("learning.txt",'r') as file:
    c=file.readline()
    while c:
      print("Query: ",c)
      tag=input("Enter its Tag:")
      response=input("Enter its response:")
      dict1["tag"]=tag
      dict1["patterns"]=list([c[:-1]])
      dict1["responses"]=list([response])
      write_json(dict1)
      dict1.clear()
      c=file.readline()


